In [59]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import enchant
import numpy as np
from collections import Counter
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import random
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import emoji
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords



#nltk.download('punkt_tab')
#nltk.download("words")
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')
#%matplotlib qt
#nltk.download('stopwords')
%matplotlib inline

In [60]:
df = pd.read_csv("scitweets_export.tsv", sep = "\t")

In [61]:
df

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0,316669998137483264,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1,319090866545385472,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1135,1255,1340455669443350528,@LaylaFanucci @realDonaldTrump I'm sorry but w...,1,1.0,0.0,0.0
1136,1256,1340689510569549824,"Dear #NIN applicants, you can kindly download ...",0,0.0,0.0,0.0
1137,1258,1341155832793165825,Whats the uber support team email address?,0,0.0,0.0,0.0
1138,1259,1344167355648241664,House passes bill to increase stimulus checks ...,0,0.0,0.0,0.0


In [62]:
text = df["text"]

In [63]:
negation_vocab = ["no", "not", "didnt", "didn't", "never"]

In [64]:
stop_words = set(stopwords.words('english'))

In [65]:
text_data_cleaned = [
    [word for word in sentence.split() if word.lower() not in stop_words]
    for sentence in text
]

In [66]:
def remake_sentences(sentence_list):
    sentence_join = []
    phrase = ""
    for sentence in sentence_list:
        for word in sentence:
            phrase = phrase + word + " "
        sentence_join.append(phrase)
        phrase = ""
    return(sentence_join)

In [ ]:
#Créer une liste des phrases ou chaque phrase est une liste de mots.
fin_text = remake_sentences(text_data_cleaned)

text_sep = [word_tokenize(t) for t in text_data

In [88]:
#Créer une liste des phrases ou chaque phrase est une liste de mots.
fin_text = remake_sentences(text_data_cleaned)

text_sep = [word_tokenize(t) for t in fin_text]

def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjectif
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verbe
        return wordnet.VERB
    elif tag.startswith('N'):  # Nom
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverbe
        return wordnet.ADV
    else:
        return wordnet.NOUN 

    
lemmatizer = WordNetLemmatizer()
refined_tweets = []
for tweet in text_data_cleaned:
    pos_tags = pos_tag(tweet)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    refined_tweets.append(lemmatized_words)



In [89]:
refined_tweets

[['Knees',
  'bite',
  'sore.',
  'guess',
  "that's",
  'sign',
  'recent',
  'treadmilling',
  'work'],
 ["McDonald's", 'breakfast', 'stop', 'gym', '🏀💪'],
 ['Gynecologist',
  'Cancer',
  'Experience',
  'explain',
  'danger',
  'Transvaginal',
  'Douching',
  'Fluoride',
  'toxins',
  'Dioxin?',
  '#PDX'],
 ['Couch-lock', 'high', 'lead', 'sleep', 'couch.', 'Gotta', 'stop', 'shit.'],
 ['daily',
  'routine',
  'help',
  'prevent',
  'problem',
  'bipolar',
  'disorder',
  'http://t.co/XGUfUDoLJB'],
 ['Impact', 'Infertility', 'Relationship', 'http://t.co/7pF7Z0wH8f'],
 ['“Traffic',
  'Jam”',
  'Brain’s',
  'Neurons',
  'Could',
  'Cause',
  'Statin-Related',
  'Cognitive',
  'Decline',
  'http://t.co/kQ3XPdePKX'],
 ['play',
  'game',
  'improve',
  'life',
  'save',
  'world?',
  '@chris_pelle',
  'via',
  '@PostArcade',
  'http://t.co/uEtVu099Ad',
  '#CDA30'],
 ['effect',
  'climate',
  'change',
  'iceberg',
  'production',
  'Greenland',
  'glacier',
  'http://t.co/CtnPhcmD7c'],
 ['@

In [75]:
tfidf = TfidfVectorizer(max_features=100000)  # tu choisis max_features selon ton cas
X_text = tfidf.fit_transform(remake_sentences(refined_tweets))
y = df["science_related"]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [76]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
x = tfidf.fit_transform(df["text"])

# Faire 5-fold cross-validation
scores_lr = cross_val_score(logreg, X_text, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.83333333 0.79824561 0.79824561 0.81578947 0.85087719 0.73684211
 0.83333333 0.75438596 0.85087719 0.87719298]
Score moyen: 0.8149122807017543


### Enlever (.,\\:';"“”’[]), les chiffres et les liens (http) de chaque tweet

In [51]:
text = []
all_text = list(df["text"])
for t in all_text:
    a = re.sub("[\.,/\\:;\"\'0-9“”’\[\]]", " ", t)
    a = re.sub("http\S*", "", a) #Enlever les liens
    #a = re.sub("#\S*", "", a) #Enlever les tags
    a = a.lower()
    text.append(a)
#for sentence in text:  

In [52]:
text = df["text"]

## Standardizer chaque phrases. 
- Convertir les noms pluriels en leur forme singuliers
- Convertir les conjugaison de chaque verbe en leur infinitif
- Standardiser les ajectives et les adverbes

In [53]:
#Créer une liste des phrases ou chaque phrase est une liste de mots.
text_sep = [word_tokenize(t) for t in text]

def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjectif
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verbe
        return wordnet.VERB
    elif tag.startswith('N'):  # Nom
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverbe
        return wordnet.ADV
    else:
        return wordnet.NOUN 

    
lemmatizer = WordNetLemmatizer()
refined_tweets = []
for tweet in text_sep:
    pos_tags = pos_tag(tweet)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    refined_tweets.append(lemmatized_words)



In [54]:
df["sep_text"] = refined_tweets

In [55]:
def remake_sentences(sentence_list):
    sentence_join = []
    phrase = ""
    for sentence in sentence_list:
        for word in sentence:
            phrase = phrase + word + " "
        sentence_join.append(phrase)
        phrase = ""
    return(sentence_join)

In [56]:
remake_sentences(df["sep_text"])[762]

'The rain didn ’ t put u off ! ( Much ) Delivery of 2018 annual constituency report to Renfrewshire North & West in # Erskine @ theSNP http : //t.co/PcCZTeT61s '

## Laisser une trace pour les emojis

In [57]:
emoji_list = []
has_emoji = False 
for sentence in refined_tweets:
    has_emoji = False
    for word in sentence:
        for char in word:
            if emoji.is_emoji(char):
                has_emoji = True
                break
    if has_emoji is True:
        emoji_list.append(1)
    else:
        emoji_list.append(0)
df["has_emoji"] = emoji_list

In [58]:
hashtag_list = []
has_hashtag = False 
for sentence in refined_tweets:
    has_hashtag = False
    for word in sentence:
        for char in word:
            if char == "#":
                has_hashtag = True
                break
    if has_hashtag is True:
        hashtag_list.append(1)
    else:
        hashtag_list.append(0)
df["has_hashtag"] = hashtag_list

In [59]:
questionlist = []
has_question = False 
for question in refined_tweets:
    has_question = False
    for word in sentence:
        for char in word:
            if char == "?":
                has_question = True
                break
    if has_question is True:
        questionlist.append(1)
    else:
        questionlist.append(0)
df["question"] = questionlist

In [60]:
questionlist = []
for sents in df["text"]:
    if "?" in sents:
        questionlist.append(1)
    else:
        questionlist.append(0)

len(questionlist)

1140

In [63]:
df

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context,sep_text,has_emoji,has_hashtag,question
0,0,316669998137483264,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0,"[Knees, be, a, bit, sore, ., i, guess, that, '...",0,0,0
1,1,319090866545385472,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0,"[McDonald, 's, breakfast, stop, then, the, gym...",1,0,0
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0,"[Can, any, Gynecologist, with, Cancer, Experie...",0,1,0
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0,"[Couch-lock, high, lead, to, sleep, in, the, c...",0,0,0
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0,"[Does, daily, routine, help, prevent, problem,...",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1135,1255,1340455669443350528,@LaylaFanucci @realDonaldTrump I'm sorry but w...,1,1.0,0.0,0.0,"[@, LaylaFanucci, @, realDonaldTrump, I, 'm, s...",0,0,0
1136,1256,1340689510569549824,"Dear #NIN applicants, you can kindly download ...",0,0.0,0.0,0.0,"[Dear, #, NIN, applicant, ,, you, can, kindly,...",1,1,0
1137,1258,1341155832793165825,Whats the uber support team email address?,0,0.0,0.0,0.0,"[Whats, the, uber, support, team, email, addre...",0,0,0
1138,1259,1344167355648241664,House passes bill to increase stimulus checks ...,0,0.0,0.0,0.0,"[House, pass, bill, to, increase, stimulus, ch...",0,0,0


In [61]:
a = 24
print(df.iloc[a]["sep_text"])
print(df.iloc[a]["question"])

['A', 'wise', 'Physician', 'say', ',', '“', 'The', 'best', 'medicine', 'for', 'Humans', 'be', 'LOVE.', '”', 'Some1', 'ask', ',', '“', 'If', 'it', 'doesn', '’', 't', 'work', '?', '”', 'He', 'smile', '&', 'answer', ',', '“', 'Increase', 'the', 'dose', '.', '”']
0


# Créations des modèles avec en utilisant que le texte 

In [62]:
tfidf = TfidfVectorizer(max_features=100000)  # tu choisis max_features selon ton cas
X_text = tfidf.fit_transform(remake_sentences(df["sep_text"]))
y = df["science_related"]

kf = KFold(n_splits=10, shuffle=True, random_state=41)

In [94]:
rf = RandomForestClassifier(class_weight='balanced')

# Faire 5-fold cross-validation
scores_rf = cross_val_score(rf, X_text, y, cv=kf, scoring='accuracy')

print("Scores par fold (Random Forest):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (Random Forest): [0.71929825 0.8245614  0.78070175 0.72807018 0.78070175 0.84210526
 0.75438596 0.77192982 0.78070175 0.79824561]
Score moyen: 0.7780701754385964


In [95]:
svm = SVC(class_weight = 'balanced')

scores_svm = cross_val_score(svm, X_text, y, cv=kf, scoring='accuracy')

print("Scores par fold (SVM):", scores_svm)
print("Score moyen:", scores_svm.mean())

Scores par fold (SVM): [0.76315789 0.85087719 0.77192982 0.75438596 0.78947368 0.80701754
 0.78947368 0.78070175 0.81578947 0.81578947]
Score moyen: 0.793859649122807


In [96]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_text, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.81578947 0.85087719 0.78947368 0.81578947 0.78947368 0.80701754
 0.79824561 0.81578947 0.85964912 0.85087719]
Score moyen: 0.819298245614035


# Créations des modèles avec les feature d'emoji et de texte

In [97]:
X_emoji = df[['has_emoji']].values
X_combined = hstack([X_text, X_emoji])

In [98]:
rf = RandomForestClassifier(class_weight='balanced')

# Faire 5-fold cross-validation
scores_rf = cross_val_score(rf, X_combined, y, cv=kf, scoring='accuracy')

print("Scores par fold (Random Forest):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (Random Forest): [0.78070175 0.81578947 0.72807018 0.73684211 0.78947368 0.8245614
 0.74561404 0.76315789 0.77192982 0.78947368]
Score moyen: 0.7745614035087719


In [99]:
svm = SVC(class_weight = 'balanced')

scores_svm = cross_val_score(svm, X_combined, y, cv=kf, scoring='accuracy')

print("Scores par fold (SVM):", scores_svm)
print("Score moyen:", scores_svm.mean())

Scores par fold (SVM): [0.80701754 0.85087719 0.78947368 0.76315789 0.79824561 0.78947368
 0.80701754 0.79824561 0.8245614  0.86842105]
Score moyen: 0.8096491228070176


In [100]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_combined, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.78947368 0.85964912 0.80701754 0.81578947 0.79824561 0.79824561
 0.85087719 0.81578947 0.8245614  0.84210526]
Score moyen: 0.8201754385964911


# Créations des modèles avec les feature de texte, d'emoji, et de hashtag

In [101]:
X_hashtag = df[['has_hashtag']].values
X_emoji_hash = hstack([X_text, X_emoji, X_hashtag])

In [102]:
rf = RandomForestClassifier(class_weight='balanced')

# Faire 5-fold cross-validation
scores_rf = cross_val_score(rf, X_emoji_hash, y, cv=kf, scoring='accuracy')

print("Scores par fold (Random Forest):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (Random Forest): [0.72807018 0.81578947 0.78070175 0.75438596 0.75438596 0.83333333
 0.76315789 0.78947368 0.78947368 0.8245614 ]
Score moyen: 0.7833333333333334


In [103]:
svm = SVC(class_weight = 'balanced')

scores_svm = cross_val_score(svm, X_emoji_hash, y, cv=kf, scoring='accuracy')

print("Scores par fold (SVM):", scores_svm)
print("Score moyen:", scores_svm.mean())

Scores par fold (SVM): [0.78070175 0.84210526 0.77192982 0.80701754 0.8245614  0.80701754
 0.81578947 0.79824561 0.84210526 0.87719298]
Score moyen: 0.8166666666666667


In [104]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_emoji_hash, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.79824561 0.85964912 0.80701754 0.80701754 0.79824561 0.80701754
 0.83333333 0.80701754 0.8245614  0.84210526]
Score moyen: 0.8184210526315789


# Création des nouvelles colonnes

In [105]:
# Liste des parties du corps
body_parts = ['hand', 'head', 'leg', 'arm', 'brain', 'heart', 'eye', 'ear', 'foot', 'knee', 'back', 'shoulder', 'joint', 'stomach', 'muscle']

# Liste des professions scientifiques
science_professions = ['scientist', 'researcher', 'biologist', 'physicist', 'chemist', 'engineer', 'astronomer', 'doctor', 'professor', 'therapist', 'physician']


In [106]:
def contains_bodypart(word_list):
    return int(any(word in body_parts for word in word_list))

df['has_bodypart'] = df['sep_text'].apply(contains_bodypart)

def contains_science_profession(word_list):
    return int(any(word in science_professions for word in word_list))

df['has_science_profession'] = df['sep_text'].apply(contains_science_profession)

In [107]:
X_haspb = df[['has_bodypart']].values
X_hasprof = df[['has_science_profession']].values
X_text_hasbp = hstack([X_text, X_haspb, X_hasprof])

In [108]:
rf = RandomForestClassifier(class_weight='balanced')

# Faire 5-fold cross-validation
scores_rf = cross_val_score(rf, X_text_hasbp, y, cv=kf, scoring='accuracy')

print("Scores par fold (Random Forest):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (Random Forest): [0.72807018 0.83333333 0.76315789 0.72807018 0.78070175 0.81578947
 0.77192982 0.80701754 0.78947368 0.81578947]
Score moyen: 0.7833333333333334


In [109]:
svm = SVC(class_weight = 'balanced')

scores_svm = cross_val_score(svm, X_text_hasbp, y, cv=kf, scoring='accuracy')

print("Scores par fold (SVM):", scores_svm)
print("Score moyen:", scores_svm.mean())

Scores par fold (SVM): [0.78070175 0.83333333 0.77192982 0.76315789 0.78070175 0.80701754
 0.79824561 0.78070175 0.8245614  0.85087719]
Score moyen: 0.7991228070175439


In [110]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_text_hasbp, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.83333333 0.84210526 0.78947368 0.81578947 0.79824561 0.79824561
 0.79824561 0.81578947 0.86842105 0.85087719]
Score moyen: 0.8210526315789473


# Création des modèles en utilisant que le texte original

In [111]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
x = tfidf.fit_transform(df["text"])

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, x, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.81578947 0.85087719 0.77192982 0.81578947 0.78947368 0.83333333
 0.84210526 0.79824561 0.85964912 0.81578947]
Score moyen: 0.8192982456140351


In [112]:
rf = RandomForestClassifier(class_weight='balanced')

# Faire 5-fold cross-validation
scores_rf = cross_val_score(rf, x, y, cv=kf, scoring='accuracy')

print("Scores par fold (Random Forest):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (Random Forest): [0.72807018 0.81578947 0.71052632 0.77192982 0.77192982 0.8245614
 0.76315789 0.78947368 0.78070175 0.78947368]
Score moyen: 0.7745614035087719


In [113]:
svm = SVC(class_weight = 'balanced')

scores_svm = cross_val_score(svm, x, y, cv=kf, scoring='accuracy')

print("Scores par fold (SVM):", scores_svm)
print("Score moyen:", scores_svm.mean())

Scores par fold (SVM): [0.77192982 0.83333333 0.77192982 0.76315789 0.80701754 0.78070175
 0.78947368 0.79824561 0.78070175 0.79824561]
Score moyen: 0.7894736842105263


In [114]:
len(df[df["has_emoji"]==1])/len(df[df["science_related"] == 0])

0.1084967320261438

# ?

In [64]:
X_q = df[['question']].values
X_text_q = hstack([X_text, X_q])

In [67]:
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
x = tfidf.fit_transform(df["text"])

# Faire 5-fold cross-validation
scores_rf = cross_val_score(logreg, X_text_q, y, cv=kf, scoring='accuracy')

print("Scores par fold (logisitic regression):", scores_rf)
print("Score moyen:", scores_rf.mean())

Scores par fold (logisitic regression): [0.81578947 0.85087719 0.78947368 0.81578947 0.78947368 0.80701754
 0.79824561 0.81578947 0.85964912 0.85087719]
Score moyen: 0.819298245614035
